In [ ]:
# !pip install torch torchvision
# !pip install opencv-contrib-python

In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import models
from torchvision import transforms, datasets

from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

In [ ]:
import cv2
import argparse
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
torch.manual_seed(64)

In [ ]:
home_path = "/content/drive/MyDrive/AllTalentZ"

In [ ]:
os.makedirs(f"{home_path}/models/train", exist_ok=True)

In [ ]:
# dir(models)

In [ ]:
# help(models.resnet152)

In [ ]:
pretrained_model = models.resnet50(weights='IMAGENET1K_V2')# pretrained=True,
num_ftrs = pretrained_model.fc.in_features

# pretrained_model = models.vgg19(weights='IMAGENET1K_V1') #pretrained=True,
# num_ftrs = pretrained_model.classifier[0].in_features
num_ftrs

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 161MB/s]


2048

In [ ]:
data_path = "/content/drive/MyDrive/AllTalentZ/data/Chess"
tmp_path = "/content/drive/MyDrive/AllTalentZ/data/Chess/train"

In [ ]:
new_num_classes = len(os.listdir(tmp_path))
new_num_classes

6

In [ ]:
# Training Config
num_epochs = 150
batch_size = 32 #64
num_workers = 2

In [ ]:
# # Freeze all the layers of the pre-trained model

for name, param in pretrained_model.named_parameters():
  if "fc" in name:
    param.requires_grad = True
  else:
    param.requires_grad = False

# for name, param in pretrained_model.named_parameters():
#   if "classifier" in name:
#     param.requires_grad = True
#   else:
#     param.requires_grad = False

In [ ]:
# Additional linear layer and dropout layer
# pretrained_model.fc
# pretrained_model.fc = nn.Sequential(
#     nn.Linear(num_ftrs, 512),
#     nn.ReLU(inplace=True),
#     nn.Dropout(0.3),
#     nn.Linear(512, 256),
#     nn.ReLU(inplace=True),
#     nn.Dropout(0.3),
#     nn.Linear(256, 128),
#     nn.ReLU(inplace=True),
#     nn.Dropout(0.3),
#     nn.Linear(128, new_num_classes)
# )

pretrained_model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 1024),
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    nn.Linear(1024, new_num_classes)
)
# pretrained_model.fc = nn.Linear(num_ftrs, new_num_classes)

In [ ]:
# pretrained_model.classifier = nn.Sequential(
#     nn.Linear(num_ftrs, 512),
#     nn.ReLU(inplace=True),
#     nn.Dropout(0.3),
#     nn.Linear(512, new_num_classes)
# )

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pretrained_model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
# Define your optimizer with weight decay
optimizer = optim.SGD(pretrained_model.parameters(), lr=0.003, momentum=0.9, weight_decay=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.3)

In [ ]:
# optimizer = optim.Adam(pretrained_model.parameters(), lr=0.003, weight_decay=0.0001)

In [ ]:
# Define the loss function - For classification problem
loss_function = nn.CrossEntropyLoss()

In [ ]:
IMAGE_SIZE = 224
data_transforms = {
    "train":transforms.Compose([
              transforms.RandomResizedCrop(size=(224,224)),
              transforms.RandomRotation(30),
              transforms.RandomHorizontalFlip(p=1),
              transforms.RandomVerticalFlip(p=1),
              # transforms.ColorJitter(brightness=(0.1, 0.3), contrast=(0.1, 0.3), saturation=(0.1, 0.3), hue=(0.1, 0.3)),
              transforms.RandomAffine(degrees=(30, 70), translate=(0.0, 0.1), scale=(0.5, 0.7)),
              transforms.ToTensor(),
              transforms.Normalize(
              mean=[0.485, 0.456, 0.406],
              std=[0.229, 0.224, 0.225]
          )]),
    "val": transforms.Compose([
              transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
              transforms.ToTensor(),
              transforms.Normalize(
              mean=[0.485, 0.456, 0.406],
              std=[0.229, 0.224, 0.225]
          )])
}


In [ ]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_path, x), data_transforms[x]) for x in ['train', 'val']}

In [ ]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=num_workers) for x in ['train', 'val']}
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x78d69a563460>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x78d69a560400>}

In [ ]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
dataset_sizes

{'train': 451, 'val': 54}

In [ ]:
class_names = image_datasets['train'].classes
class_names

['Bishop', 'King', 'Knight', 'Pawn', 'Queen', 'Rook']

In [ ]:
train_losses = []  # To store the losses for plotting
val_losses = []  # To store the losses for plotting
best_val_loss = float('inf')  # Initialize with a very large value
best_val_acc = 0
use_val_acc = True

In [ ]:
# Train the model
for epoch in range(num_epochs):

    # Train the model on the training set
    pretrained_model.train()
    # Initialize the training loss accumulator to zero
    training_loss = 0.0

    for i, (image, labels) in enumerate(dataloaders["train"]):
        # Prepare data and send it to the proper device
        image = image.to(device)
        labels = labels.to(device)

        # Clear the gradients of all optimized parameters
        optimizer.zero_grad()

        # Forward pass: obtain model predictions for the input data
        outputs = pretrained_model(image)
        loss = loss_function(outputs, labels)

        # Backward pass: compute gradients of the loss with respect to model parameters
        loss.backward()

        # Update model parameters using the computed gradients and the optimizer
        optimizer.step()

        # Update the training loss
        training_loss += loss.item() * image.size(0)

    # Calculate average training loss
    train_loss = training_loss / dataset_sizes["train"]
    train_losses.append(train_loss)
    scheduler.step()

    # Evaluate the model on the validation set
    pretrained_model.eval()
    val_loss = 0.0
    correct_preds = 0
    total_samples = 0
    all_labels = []
    all_pred = []
    with torch.no_grad():
        for image, labels in dataloaders["val"]:
            # Prepare data and send it to the proper device
            image = image.to(device)
            labels = labels.to(device)

            # Forward pass: obtain model predictions for the input data
            outputs = pretrained_model(image)
            _, predicted = torch.max(outputs, 1)

            # Compute the loss between the model predictions and the true labels
            loss = loss_function(outputs, labels)

            # Update the validation loss
            val_loss += loss.item() * image.size(0)

            # Round up and down to either 1 or 0
            total_samples += labels.size(0)
            # Calculate how many images were correctly classified
            predicted =  predicted.view(1, -1)
            labels = labels.view(1, -1)
            correct_preds += torch.sum(torch.eq(predicted, labels), dim=1).item()

            all_labels.extend(labels.cpu().numpy())
            all_pred.extend(predicted.cpu().numpy())

    # Calculate validation loss
    val_loss /= dataset_sizes["val"]
    val_losses.append(val_loss) # To store the losses for plotting

    # Calculate validation accuracy
    val_acc = correct_preds / total_samples #* 100

    # Print validation loss and accuracy
    print(f"Epoch [{epoch + 1}/{num_epochs}] Train Loss: {train_loss:.4f}  Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%")

    # Save the model if it performs better on validation set

    if use_val_acc:
      if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(pretrained_model.state_dict(), f'/content/drive/MyDrive/AllTalentZ/models/train/best_model_epoch_{epoch + 1}--val_acc_{best_val_acc:.2f}.pth')
    else:
      if val_loss < best_val_loss:
          best_val_loss = val_loss
          torch.save(pretrained_model.state_dict(), f'/content/drive/MyDrive/AllTalentZ/models/train/best_model_epoch_{epoch + 1}--val_loss_{best_val_loss:.4f}.pth')

print('Finished Training')

Epoch [1/150] Train Loss: 1.6131  Validation Loss: 1.6518, Validation Accuracy: 0.41%
Epoch [2/150] Train Loss: 1.5854  Validation Loss: 1.6432, Validation Accuracy: 0.39%
Epoch [3/150] Train Loss: 1.5995  Validation Loss: 1.6638, Validation Accuracy: 0.46%
Epoch [4/150] Train Loss: 1.6068  Validation Loss: 1.6715, Validation Accuracy: 0.43%
Epoch [5/150] Train Loss: 1.5982  Validation Loss: 1.6698, Validation Accuracy: 0.44%
Epoch [6/150] Train Loss: 1.5795  Validation Loss: 1.6750, Validation Accuracy: 0.44%
Epoch [7/150] Train Loss: 1.6047  Validation Loss: 1.6648, Validation Accuracy: 0.41%
Epoch [8/150] Train Loss: 1.5822  Validation Loss: 1.6503, Validation Accuracy: 0.46%
Epoch [9/150] Train Loss: 1.6013  Validation Loss: 1.6733, Validation Accuracy: 0.44%
Epoch [10/150] Train Loss: 1.5783  Validation Loss: 1.6444, Validation Accuracy: 0.46%
Epoch [11/150] Train Loss: 1.5977  Validation Loss: 1.6457, Validation Accuracy: 0.44%
Epoch [12/150] Train Loss: 1.5946  Validation Loss: 

In [ ]:
# Plotting the evolution of loss
plt.plot(train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Evolution of Training Loss')
plt.legend()
plt.show()

In [ ]:
# Plotting the evolution of loss
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Evolution of Validation Loss')
plt.legend()
plt.show()

In [ ]:
# Calculate metrics
predicted = predicted.cpu().numpy()
labels =  labels.cpu().numpy()